In [1]:
import os 
import math
import uproot
import numpy as np
import h5py
import sys 
import tensorflow_io as tfio
import tensorflow as tf

2021-07-12 15:46:31.535913: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2021-07-12 15:46:31.536009: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#from skhep.math import LorentzVector

In [3]:
file_in = uproot.open("/eos/home-t/thaarres/L1_tuple/perfNano_TTbar_PU200_GenParts_More.root")
tree = file_in.get("Events")
branches = tree.arrays()

In [4]:
nJets = branches["nL1PuppiSC4Jets"]
print(len(nJets))

119392


In [5]:
def getParticle(branches, label, iEvt):
    features = ["_pt", "_eta", "_phi", "_mass", "_pdgId", "_vz"]
    cands =  np.array([])
    outNames = []
    for i in range(30):
        this_cand =  np.array([])
        for f in features:
            f_name = "%s_dau%i%s" %(label, i, f)
            outNames.append(f_name)
            my_column = np.array(branches[f_name][iEvt])
            my_column = np.reshape(my_column, (my_column.shape[0],1,1))
            this_cand = np.concatenate((this_cand, my_column), axis=2) if this_cand.size else my_column
        cands = np.concatenate((cands, this_cand), axis=1) if cands.size else this_cand
    return cands, outNames

def findGenJet(recoJet, genJets):
    idx = -99
    for i, genJet in enumerate(genJets):
        if genJet[0] == recoJet[4]:
            idx = i

    return idx

def getJet(branches, label, iEvt, gen=False):
    features = ["_pt", "_eta", "_phi", "_mass", "_genpt", "_gendr"]
    if gen == True: 
        features = ["_pt", "_eta", "_phi", "_mass", "_partonFlavour", "_hadronFlavour"]
    for i in range(len(features)):
        features[i] = "%s%s" %(label, features[i])
    jets = np.array([])
    for f in features:
        my_column = np.array(branches[f][iEvt])
        my_column = np.reshape(my_column, (my_column.shape[0],1))
        jets = np.concatenate((jets, my_column), axis=1) if jets.size else my_column
    if gen == False:
        features.append("{}_partonFlavour".format(label))
        features.append("{}_hadronFlavour".format(label))
        features.append("{}_truth".format(label))
        for i in range(0,3):
            my_column = np.zeros(len(branches[f][iEvt]))
            my_column = np.reshape(my_column, (my_column.shape[0],1))
            jets = np.concatenate((jets, my_column), axis=1) if jets.size else my_column
        
    return jets, features

In [ ]:
recoJets_all = np.array([])
recoCands_all = np.array([])

recoJetFeatures = []
recoCandFeatures = []

for i in range(0, len(nJets)):

    recoJets, recoJetFeatures   = getJet(branches, "L1PuppiSC4Jets", i) #L1PuppiSC4Jets, L1PuppiSC4EmuJets
    recoCands, recoCandFeatures = getParticle(branches, "L1PuppiSC4Jets", i)
    genJets, genJetFeatures     = getJet(branches, "GenJets", i, True)
    
    if recoJets.shape[0] == 0 or genJets.shape[0] == 0: 
        continue

    for j in range(recoJets.shape[0]):
        
        match_idx = findGenJet(recoJets[j], genJets)
        if match_idx == -99:
            recoJets[j][6] = 0
            recoJets[j][7] = 0
            recoJets[j][8] = 0
        else:
            genMatch = genJets[match_idx,:]
            
            partonFlavour  = genJets[match_idx,4]
            hadronFlavour  = genJets[match_idx,5]
            
            recoJets[j][6] = partonFlavour
            recoJets[j][7] = hadronFlavour
        
            # Is b
            if math.fabs(partonFlavour) == 5 and math.fabs(hadronFlavour) == 5 :
                recoJets[j][8] = int(1)
            
            # Is light
            elif 0<math.fabs(partonFlavour)<5 and math.fabs(hadronFlavour) == 0 :
                recoJets[j][8] = int(2)
            
            # Is gluon
            elif math.fabs(partonFlavour) == 21 and math.fabs(hadronFlavour) == 0 :
                recoJets[j][8] = int(3)        
            else:
                recoJets[j][8] = int(0)
    recoJets_all  = np.concatenate((recoJets_all, recoJets),  axis = 0) if recoJets_all.size else recoJets
    recoCands_all = np.concatenate((recoCands_all, recoCands), axis = 0) if recoCands_all.size else recoCands

print("ALL Reco jets", recoJets_all.shape)
print("ALL Reco Cands", recoCands_all.shape)

outFile = h5py.File("ttbar_mini.h5", "w")

outFile.create_dataset('recoJets', data=recoJets_all, compression='gzip', dtype='f')
outFile.create_dataset('recoCands', data=recoCands_all, compression='gzip', dtype='f')
outFile.create_dataset('labels', data=recoJets_all[:,-1].T, compression='gzip', dtype='i')
recoJetFeatures = [st.encode('utf8') for st in recoJetFeatures] 
outFile.create_dataset('recoJetFeatureNames', data=recoJetFeatures, compression='gzip')
recoCandFeatures = [st.encode('utf8') for st in recoCandFeatures]
outFile.create_dataset('recoCandFeatureNames',  data=recoCandFeatures, compression='gzip')

outFile.close()
    

In [ ]:
print('Number of b-quarks = {}'    .format(len(recoJets_all[(recoJets_all[:,8])==1])))
print('Number of light quarks = {}'.format(len(recoJets_all[(recoJets_all[:,8])==2])))
print('Number of gluons = {}'      .format(len(recoJets_all[(recoJets_all[:,8])==3])))
print('Undefined = {}'             .format(len(recoJets_all[(recoJets_all[:,8])==0])))

In [ ]:
with h5py.File('ttbar_mini.h5', "r") as f:
    
    x_jets = np.array(f['recoJets'])
    x_train = np.array(f['recoCands'])
    y_train = tf.one_hot(f['labels'], 4)

# TODO! Transate to TF data, something weird happens to the labels?!
# # Create an IODataset from a hdf5 file's dataset object  
# x = tfio.IODataset.from_hdf5('ttbar.h5', dataset='/recoCands')
# y = tfio.IODataset.from_hdf5('ttbar.h5', dataset='/labels',dtype=tf.uint8)
    
# Zip together samples and corresponding labels
# train_ds = tf.data.Dataset.zip((x,y))
# for example in train_ds.take(1):
#     break
# print("X train batch shape = {}, Y train batch shape = {} ".format(example[0].shape, example[1].shape))
# print(example[0])
# print(example[1])

# def preprocess(image, label,nclasses=10):
#   label = tf.one_hot(tf.squeeze(label), nclasses, dtype=tf.int8)
#   return image, label

# train = train_ds.map(preprocess,4)
# train = train.batch(1000, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.regularizers import l1
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

from tensorflow.keras.models import Model

input_shape = (x_train.shape[1],x_train.shape[2],1)
n_classes = 4
print(input_shape)
filters_per_conv_layer = [16,16,24]
neurons_per_dense_layer = [42,64]

x = x_in = Input(input_shape)
x = BatchNormalization()(x)

for i,f in enumerate(filters_per_conv_layer):
    print( ('Adding convolutional block {} with N={} filters').format(i,f) )
    x = Conv2D(int(f), kernel_size=(2,2), strides=(1,1), kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=False,
               name='conv_{}'.format(i))(x) 
    x = BatchNormalization(name='bn_conv_{}'.format(i))(x)
    x = Activation('relu',name='conv_act_%i'%i)(x)
#     x = MaxPooling2D(pool_size = (2,1),name='pool_{}'.format(i) )(x)
x = Flatten()(x)

for i,n in enumerate(neurons_per_dense_layer):
  print( ('Adding dense block {} with N={} neurons').format(i,n) )
  x = Dense(n,kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001),name='dense_%i'%i, use_bias=False)(x)
  x = BatchNormalization(name='bn_dense_{}'.format(i))(x)
  x = Activation('relu',name='dense_act_%i'%i)(x)
x = Dense(int(n_classes),name='output_dense')(x)
x_out = Activation('softmax',name='output_softmax')(x)

model = Model(inputs=[x_in], outputs=[x_out], name='keras_baseline')

model.summary()

In [ ]:
LOSS        = tf.keras.losses.CategoricalCrossentropy()
OPTIMIZER   = tf.keras.optimizers.Adam(learning_rate=3E-3, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True)

model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=["accuracy"])

callbacks = [
            tf.keras.callbacks.EarlyStopping(patience=10, verbose=1),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
            ] 

model.fit(x_train,y_train,
                     epochs = 100,
                     validation_split=0.2,
                     callbacks = callbacks)   